# Demonstrate Seq2Seq Wrapper with twitter chat log

In [6]:
import tensorflow as tf
import numpy as np

# preprocessed data
from datasets.twitter import data
import data_utils

In [7]:
# load data from pickle and npy files
metadata, idx_q, idx_a = data.load_data(PATH='datasets/twitter/')
print(idx_q)
(trainX, trainY), (testX, testY), (validX, validY) = data_utils.split_dataset(idx_q, idx_a)


[[   7  106    3 ...,    0    0    0]
 [ 100    4  467 ..., 3840   16 1585]
 [   1    1 1741 ...,    0    0    0]
 ..., 
 [ 855  313   32 ..., 1071 1989  316]
 [2386  145   74 ...,    0    0    0]
 [7867  606  316 ...,    0    0    0]]


In [8]:
# parameters 
xseq_len = trainX.shape[-1]
yseq_len = trainY.shape[-1]
batch_size = 16
xvocab_size = len(metadata['idx2w'])  
yvocab_size = xvocab_size
emb_dim = 1024

In [9]:
import seq2seq_wrapper

In [10]:
import importlib
importlib.reload(seq2seq_wrapper)

<module 'seq2seq_wrapper' from '/home/joshua/Radboud/CCMLWI/wg4-chatbot/CCMLWI_Chatbot/seq2seq/seq2seq_wrapper.py'>

In [11]:
model = seq2seq_wrapper.Seq2Seq(xseq_len=xseq_len,
                               yseq_len=yseq_len,
                               xvocab_size=xvocab_size,
                               yvocab_size=yvocab_size,
                               ckpt_path='ckpt/twitter/',
                               emb_dim=emb_dim,
                               num_layers=3
                               )

<log> Building Graph </log>

In [12]:
val_batch_gen = data_utils.rand_batch_gen(validX, validY, 256)
test_batch_gen = data_utils.rand_batch_gen(testX, testY, 256)
train_batch_gen = data_utils.rand_batch_gen(trainX, trainY, batch_size)

In [8]:
#sess = model.train(train_batch_gen, val_batch_gen)

In [14]:
sess = model.restore_last_session()

In [17]:
input_ = test_batch_gen.__next__()[0]
print(input_.shape)
output = model.predict(sess, input_)
print(output.shape)

(20, 256)
(256, 20)


In [11]:
replies = []
for ii, oi in zip(input_.T, output):
    q = data_utils.decode(sequence=ii, lookup=metadata['idx2w'], separator=' ')
    decoded = data_utils.decode(sequence=oi, lookup=metadata['idx2w'], separator=' ').split(' ')
    if decoded.count('unk') == 0:
        if decoded not in replies:
            print('q : [{0}]; a : [{1}]'.format(q, ' '.join(decoded)))
            replies.append(decoded)

q : [has it been unk yet in any fest]; a : [but its not a good idea]
q : [whats going on i missed it all i hear is unk in at unk now]; a : [i think the same thing is]
q : [live with the giant unk unk come watch amp retweet for a live shoutout]; a : [this is a good thing to be the first time]
q : [cmon i wanna endorse you]; a : [is that a joke]
q : [the less you worry about what other people are saying or doing the happier you will be]; a : [i dont think its a joke but i dont think the same thing is the only thing]
q : [should i delete this tweet or let the typo stand as a joke]; a : [they dont have to be a fucking one]
q : [unk to find this on the beach this morning a unk reminder of the impact we have on our environment]; a : [is this a thing]
q : [if they worked no one would ever need to pay a unk again]; a : [maybe if you have any idea what to do with the other states]
q : [seattle rudy unk kicked the shit out of my unk girlfriend last night do not support this garbage h]; a : [you 